<a href="https://colab.research.google.com/github/OVP2023/E-CAP-2024/blob/main/ecap2_%D1%84%D0%BE%D1%80%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D0%B4%D0%B0%D1%82%D0%B0_%D1%81%D0%B5%D1%82%D0%B0_%D0%B4%D0%BB%D1%8F_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Сначала формируем дата сэт из MyDrive/ecap/train.parquet' с отмеченным целевым столбцом target  c названием товара из MyDrive/ecap/text_and_bert.parquet

In [ ]:
import pandas as pd
import numpy as np
import json


In [5]:
train = pd.read_parquet('/content/drive/MyDrive/ecap/train.parquet') #для тренироваки модели variantid1  id   товара 1   variantid2 id  товара  2   target  метка  1 матч(совпадение)  0 не матч
train.head()

,variantid1,variantid2,target
0,1447875869,1447872068,1
1,1176231201,284733670,1
2,658617865,549848659,0
3,719320625,719370486,1
4,1067645658,949954740,0


In [6]:
text_and_bert = pd.read_parquet('/content/drive/MyDrive/ecap/text_and_bert.parquet')
text_and_bert.head() #name название товара  description описание товара   name_bert_64 эмбендинг наименования товара

,variantid,name,description,name_bert_64
0,47920382,"Игрушка для ванной , сувенир Уточка Дьяволица",Серия уточек бренда FUNNY DUCKS представлена ...,"[-0.24964939057826996, 0.6433379650115967, 0.4..."
1,49801845,Стразы(бусины) клеевые на листе 9*16 см,Стразы(бусины) клеевые на листе 9*16 см,"[-0.6397916078567505, 0.3660058379173279, 0.67..."
2,49853444,"Набор для вышивания Vervaco ""Подушка. Геометри...",Состав набора: канва-страмин Zweigart с нанесе...,"[-0.27170804142951965, 0.3709857165813446, 0.4..."
3,49893028,"Кружево коклюшечное ""Prym"", цвет: серый, 8 мм,...","Кружево коклюшечное ""Prym"" предназначено для о...","[-0.5911799669265747, 0.45404374599456787, 0.4..."
4,49987483,Оригами Настольная игра Фиксики Кодовый замок,"В настольной игре Оригами ""Фиксики. Кодовый за...","[-0.29846125841140747, 0.41105785965919495, 0...."


In [8]:
#объединение train и text_and_bert   создание одного датасета     merge_data(train, text_and_bert):
train_data = train.merge(text_and_bert[['variantid', 'name_bert_64']], left_on='variantid1', right_on='variantid', how='left')
train_data = train_data.rename(columns={'name_bert_64': 'emb_name1'})
train_data = train_data.drop(columns=['variantid'])

train_data = train_data.merge(text_and_bert[['variantid', 'name_bert_64']], left_on='variantid2', right_on='variantid', how='left')
train_data = train_data.rename(columns={'name_bert_64': 'emb_name2'})
train_data = train_data.drop(columns=['variantid'])
train_data = train_data.dropna()

In [ ]:
#train_data.shape[0] #подсчитываем количество строк в train_data

1168516

In [9]:
from sklearn.metrics.pairwise import cosine_similarity

def python_cosine_similarity(lst1, lst2):  #функция косинусного сходства
    dot_product = sum(a * b for a, b in zip(lst1, lst2))  # Скалярное произведение
    norm_lst1 = sum(a**2 for a in lst1) ** 0.5  # Рассчитаем L2-норму lst1
    norm_lst2 = sum(b**2 for b in lst2) ** 0.5  # Рассчитаем L2-норму lst2
    return dot_product / (norm_lst1 * norm_lst2)

min_cos=1
for i in range(0, 1168515):
    cos_= python_cosine_similarity(train_data['emb_name1'][i],train_data['emb_name2'][i])
    if min_cos > cos_:
        min_cos=cos_
    train_data.loc[i,'Cosine_similarity_names']=cos_
print (min_cos)
train_data = train_data.drop(columns=['emb_name1'])
train_data = train_data.drop(columns=['emb_name2'])

0.839931510585819


In [10]:
train_data.head()

,variantid1,variantid2,target,Cosine_similarity_names
0,1447875869,1447872068,1,1.000000
1,1176231201,284733670,1,0.989477
2,658617865,549848659,0,0.966868
3,719320625,719370486,1,1.000000
4,1067645658,949954740,0,0.986480


In [11]:
train_data.to_parquet('/content/drive/MyDrive/ecap/mydata.parquet')